In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import os
import shutil

working_dir = "/kaggle/working"

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)

    if os.path.isfile(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

print("All contents within /kaggle/working have been deleted.")

/kaggle/input/8000-images-rfedtr-freeze-backbone/checkpoint_best_regular.pth
/kaggle/input/8000-images-rfedtr-freeze-backbone/checkpoint_best_ema.pth
/kaggle/input/8000-images-new-rfdetr-freeze-backbone/checkpoint_best_regular.pth
/kaggle/input/8000-images-new-rfdetr-freeze-backbone/checkpoint_best_ema.pth
/kaggle/input/8000-images-dataset/README.dataset.txt
/kaggle/input/8000-images-dataset/README.roboflow.txt
/kaggle/input/8000-images-dataset/data.yaml
/kaggle/input/8000-images-dataset/valid/labels/302e07eb7155dc39_jpg.rf.3ade9b6d15fad6cf248ac940123b5681.txt
/kaggle/input/8000-images-dataset/valid/labels/3f733474f5833879_jpg.rf.07c0139b4940aeadf3a0480dff74626e.txt
/kaggle/input/8000-images-dataset/valid/labels/d5ec6c4bded4c388_jpg.rf.8893b6718d4b464d2010c126577f4015.txt
/kaggle/input/8000-images-dataset/valid/labels/7df8dbc0eb50f51c_jpg.rf.68d55cd7ee71f16071f1e0f14576b9b1.txt
/kaggle/input/8000-images-dataset/valid/labels/9d015956ddf611c4_jpg.rf.96ccab6c34356e58a2ce77a99931f684.txt
/

In [2]:
!pip install pycocotools fiftyone
!pip install tqdm
!pip install -U albumentations
!pip install rfdetr torchvision pytorch-lightning
!pip install rfdetr[onnxexport]
!pip install rfdetr supervision albumentations opencv-python torch torchvision --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00

In [3]:
import os
import json
import shutil
import yaml
import cv2
import numpy as np
from PIL import Image
from rfdetr import RFDETRBase
from tqdm import tqdm
import torch
import albumentations as A

# =====================
# Configuration
# =====================
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
yolo_dir = "/kaggle/input/8000-images-dataset"
output_dir = "/kaggle/working/coco_dataset"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =====================
# Preprocessing Functions
# =====================
def adjust_brightness_contrast(image, alpha=1.2, beta=15, **kwargs):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def simple_white_balance(image, **kwargs):
    result = image.astype(np.float32)
    avg_b, avg_g, avg_r = np.mean(result, axis=(0,1))
    avg_gray = (avg_b + avg_g + avg_r) / 3
    result[..., 0] *= avg_gray / avg_b
    result[..., 1] *= avg_gray / avg_g
    result[..., 2] *= avg_gray / avg_r
    return np.clip(result, 0, 255).astype(np.uint8)

def histogram_equalization(image, **kwargs):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[..., 0] = cv2.equalizeHist(img_yuv[..., 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

preprocess = A.Compose([
    A.Lambda(image=lambda image, **kw: adjust_brightness_contrast(image)),
    A.Lambda(image=lambda image, **kw: simple_white_balance(image)),
    A.Lambda(image=lambda image, **kw: histogram_equalization(image)),
])

# =====================
# Dataset Conversion
# =====================
def process_image(image_path):
    """Apply preprocessing pipeline"""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    processed = preprocess(image=image)['image']
    return processed

def yolo_to_coco(yolo_root, output_root):
    # Load class names
    with open(os.path.join(yolo_root, "data.yaml")) as f:
        class_names = yaml.safe_load(f)['names']
    
    if len(class_names) == 1:
        class_names.append("background")

    # Create category mapping
    categories = [{"id": i, "name": n, "supercategory": "object"} for i, n in enumerate(class_names)]

    for split in ['test']:
        split_dir = os.path.join(output_root, split)
        os.makedirs(split_dir, exist_ok=True)
        
        coco_data = {
            "info": {"description": "Preprocessed Dataset"},
            "licenses": [{"name": "MIT"}],
            "categories": categories,
            "images": [],
            "annotations": []
        }

        annotation_id = 1
        image_id = 1

        # Process images with preprocessing
        img_dir = os.path.join(yolo_root, split, "images")
        lbl_dir = os.path.join(yolo_root, split, "labels")

        for img_file in tqdm(os.listdir(img_dir)):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            # Process and save image
            src_path = os.path.join(img_dir, img_file)
            dst_path = os.path.join(split_dir, img_file)
            
            # Apply preprocessing chain
            processed_img = process_image(src_path)
            cv2.imwrite(dst_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))

            # Get dimensions
            height, width = processed_img.shape[:2]

            coco_data["images"].append({
                "id": image_id,
                "file_name": img_file,
                "width": width,
                "height": height,
                "license": 1,
                "date_captured": "2024-01-01"
            })

            # Process labels
            label_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) != 5: continue

                        cls_id, xc, yc, w, h = map(float, parts)
                        x_min = (xc - w/2) * width
                        y_min = (yc - h/2) * height
                        box_w = w * width
                        box_h = h * height

                        coco_data["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": int(cls_id),
                            "bbox": [x_min, y_min, box_w, box_h],
                            "area": box_w * box_h,
                            "iscrowd": 0
                        })
                        annotation_id += 1

            image_id += 1
        with open(os.path.join(split_dir, "_annotations.coco.json"), 'w') as f:
            json.dump(coco_data, f, indent=2)

# =====================
# Training Setup
# =====================
if __name__ == "__main__":
    # Convert dataset
    import os
    import shutil
    
    working_dir = "/kaggle/working"
    
    for item in os.listdir(working_dir):
        item_path = os.path.join(working_dir, item)
    
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    
    print("All contents within /kaggle/working have been deleted.")
    yolo_to_coco(yolo_dir, output_dir)

2025-04-30 03:06:49.036952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745982409.258235      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745982409.314189      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


All contents within /kaggle/working have been deleted.


100%|██████████| 348/348 [00:08<00:00, 41.51it/s]


In [4]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import shutil
import zipfile
from PIL import Image
from scipy.optimize import linear_sum_assignment
from rfdetr import RFDETRBase
from tqdm import tqdm

# =====================
# Configuration
# =====================
COCO_TEST_DIR = "/kaggle/working/coco_dataset/test"
MODEL_PATH = "/kaggle/input/8000-images-new-rfdetr-freeze-backbone/checkpoint_best_ema.pth"
IOU_THRESHOLD = 0.5
OUTPUT_DIR = "/kaggle/working/results-ema"
ZIP_PATH = "/kaggle/working/results-ema.zip"
TILE_SIZE = (200, 200)

print("Model exists:", os.path.exists(MODEL_PATH))  # Should return True

# =====================
# Delete existing output
# =====================
for path in [OUTPUT_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

# =====================
# COCO Annotation Loader
# =====================
def load_coco_annotations(json_path):
    with open(json_path) as f:
        data = json.load(f)
    
    id_to_image = {img["id"]: img for img in data["images"]}
    annotations = {}
    
    for ann in data["annotations"]:
        image_id = ann["image_id"]
        if image_id not in annotations:
            annotations[image_id] = []
        annotations[image_id].append(ann)
    
    return id_to_image, annotations

# =====================
# IoU Calculation
# =====================
def compute_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

# =====================
# Grid Creation Function
# =====================
def create_image_grid(image_paths, grid_path):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, TILE_SIZE)
            images.append(img)
        else:
            images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    while len(images) < 16:
        images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    rows = []
    for i in range(0, 16, 4):
        row = np.hstack(images[i:i+4])
        rows.append(row)
    grid = np.vstack(rows)
    cv2.imwrite(grid_path, grid)

# =====================
# Metrics Calculation
# =====================
def calculate_metrics(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

# =====================
# Main Processing
# =====================
def analyze_predictions():
    # Initialize model
    model = RFDETRBase(checkpoint_path=MODEL_PATH)
    
    # Load COCO data
    coco_json = os.path.join(COCO_TEST_DIR, "_annotations.coco.json")
    id_to_image, coco_anns = load_coco_annotations(coco_json)
    
    # Create output directories
    predicted_dir = os.path.join(OUTPUT_DIR, "predicted")
    wrong_dir = os.path.join(OUTPUT_DIR, "wrong")
    os.makedirs(predicted_dir, exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "iou_threshold"), exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "label_mismatch"), exist_ok=True)
    
    # Metrics tracking
    global_tp = 0
    global_fp = 0
    global_fn = 0
    failure_data = []
    
    # Process each image
    for image_id in tqdm(id_to_image, desc="Processing Images"):
        img_info = id_to_image[image_id]
        img_path = os.path.join(COCO_TEST_DIR, img_info["file_name"])
        
        # Load image and get predictions
        image = Image.open(img_path)
        detections = model.predict(image)
        pred_boxes = np.array(detections.xyxy)
        
        # Get ground truth boxes
        gt_boxes = []
        if image_id in coco_anns:
            for ann in coco_anns[image_id]:
                x, y, w, h = ann["bbox"]
                gt_boxes.append([x, y, x+w, y+h])
        gt_boxes = np.array(gt_boxes)
        
        # Create annotated image
        processed_img = cv2.imread(img_path)
        filename = os.path.basename(img_path)
        
        # Draw annotations
        if gt_boxes.size > 0:
            for gt in gt_boxes:
                cv2.rectangle(processed_img, 
                            (int(gt[0]), int(gt[1])),
                            (int(gt[2]), int(gt[3])),
                            (0, 255, 0), 2)
        
        if pred_boxes.size > 0:
            for box in pred_boxes:
                cv2.rectangle(processed_img,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            (0, 0, 255), 2)
        
        # Save to predicted folder
        cv2.imwrite(os.path.join(predicted_dir, filename), processed_img)
        
        # Initialize metrics for current image
        tp = 0
        fp = 0
        fn = 0
        
        has_gt = gt_boxes.size > 0
        has_pred = pred_boxes.size > 0
        
        if has_gt and has_pred:
            cost_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))
            for i, gt in enumerate(gt_boxes):
                for j, pred in enumerate(pred_boxes):
                    cost_matrix[i, j] = -compute_iou(gt, pred)
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            
            matched_gt = set()
            matched_pred = set()
            
            for i, j in zip(row_ind, col_ind):
                iou_val = compute_iou(gt_boxes[i], pred_boxes[j])
                if iou_val >= IOU_THRESHOLD:
                    tp += 1
                    matched_gt.add(i)
                    matched_pred.add(j)
            
            fp += len(pred_boxes) - len(matched_pred)
            fn += len(gt_boxes) - len(matched_gt)
            
        elif has_pred:
            fp += len(pred_boxes)
        elif has_gt:
            fn += len(gt_boxes)
        
        # Update global metrics
        global_tp += tp
        global_fp += fp
        global_fn += fn
        
        # Failure analysis
        label_mismatch = (len(gt_boxes) != len(pred_boxes)) or (fp > 0) or (fn > 0)
        low_iou = (tp > 0) and (fp + fn > 0)
        
        # Save failure cases
        failure_reasons = []
        if label_mismatch:
            failure_reasons.append("label_mismatch")
            cv2.imwrite(os.path.join(wrong_dir, "label_mismatch", filename), processed_img)
        if low_iou:
            failure_reasons.append("iou_threshold")
            cv2.imwrite(os.path.join(wrong_dir, "iou_threshold", filename), processed_img)
        
        if failure_reasons:
            failure_data.append({
                "image": filename,
                "tp": tp,
                "fp": fp,
                "fn": fn,
                "failure_reasons": ", ".join(failure_reasons)
            })

    # Calculate final metrics
    precision, recall = calculate_metrics(global_tp, global_fp, global_fn)
    
    # Save metrics to file
    metrics_content = f"""Evaluation Metrics:
    - True Positives (TP): {global_tp}
    - False Positives (FP): {global_fp}
    - False Negatives (FN): {global_fn}
    - Precision: {precision:.4f}
    - Recall: {recall:.4f}
    - F1 Score: {2*(precision*recall)/(precision+recall):.4f}"""
    
    with open(os.path.join(OUTPUT_DIR, "metrics.txt"), "w") as f:
        f.write(metrics_content)
    
    # Save CSV report
    df = pd.DataFrame(failure_data)
    csv_path = os.path.join(OUTPUT_DIR, "failure_report.csv")
    df.to_csv(csv_path, index=False)
    
    # Create grids for wrong predictions
    for subfolder in ["iou_threshold", "label_mismatch"]:
        subfolder_path = os.path.join(wrong_dir, subfolder)
        if os.path.exists(subfolder_path):
            image_files = sorted([f for f in os.listdir(subfolder_path) 
                               if f.endswith('.jpg') and not f.startswith('grid_')])
            
            for i in range(0, len(image_files), 16):
                chunk = image_files[i:i+16]
                grid_path = os.path.join(subfolder_path, f"grid_{i//16 + 1}.jpg")
                image_paths = [os.path.join(subfolder_path, f) for f in chunk]
                create_image_grid(image_paths, grid_path)

    # Create zip archive
    shutil.make_archive(ZIP_PATH.rstrip('.zip'), 'zip', OUTPUT_DIR)
    
    print(f"\n✅ Analysis complete!")
    print(f"📦 Download results: {ZIP_PATH}")
    print(metrics_content)

if __name__ == "__main__":
    analyze_predictions()

Model exists: True


rf-detr-base.pth: 100%|██████████| 355M/355M [00:03<00:00, 101MiB/s] 


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Loading pretrain weights


Processing Images: 100%|██████████| 348/348 [00:17<00:00, 19.98it/s]



✅ Analysis complete!
📦 Download results: /kaggle/working/results-ema.zip
Evaluation Metrics:
    - True Positives (TP): 383
    - False Positives (FP): 386
    - False Negatives (FN): 102
    - Precision: 0.4980
    - Recall: 0.7897
    - F1 Score: 0.6108


In [5]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import shutil
import zipfile
from PIL import Image
from scipy.optimize import linear_sum_assignment
from rfdetr import RFDETRBase
from tqdm import tqdm

# =====================
# Configuration
# =====================
COCO_TEST_DIR = "/kaggle/working/coco_dataset/test"
MODEL_PATH = "/kaggle/input/8000-images-new-rfdetr-freeze-backbone/checkpoint_best_regular.pth"
IOU_THRESHOLD = 0.5
OUTPUT_DIR = "/kaggle/working/results-regular"
ZIP_PATH = "/kaggle/working/results-regular.zip"
TILE_SIZE = (200, 200)

print("Model exists:", os.path.exists(MODEL_PATH))  # Should return True

# =====================
# Delete existing output
# =====================
for path in [OUTPUT_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

# =====================
# COCO Annotation Loader
# =====================
def load_coco_annotations(json_path):
    with open(json_path) as f:
        data = json.load(f)
    
    id_to_image = {img["id"]: img for img in data["images"]}
    annotations = {}
    
    for ann in data["annotations"]:
        image_id = ann["image_id"]
        if image_id not in annotations:
            annotations[image_id] = []
        annotations[image_id].append(ann)
    
    return id_to_image, annotations

# =====================
# IoU Calculation
# =====================
def compute_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

# =====================
# Grid Creation Function
# =====================
def create_image_grid(image_paths, grid_path):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, TILE_SIZE)
            images.append(img)
        else:
            images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    while len(images) < 16:
        images.append(np.zeros((TILE_SIZE[1], TILE_SIZE[0], 3), dtype=np.uint8))
    
    rows = []
    for i in range(0, 16, 4):
        row = np.hstack(images[i:i+4])
        rows.append(row)
    grid = np.vstack(rows)
    cv2.imwrite(grid_path, grid)

# =====================
# Metrics Calculation
# =====================
def calculate_metrics(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

# =====================
# Main Processing
# =====================
def analyze_predictions():
    # Initialize model
    model = RFDETRBase(checkpoint_path=MODEL_PATH)
    
    # Load COCO data
    coco_json = os.path.join(COCO_TEST_DIR, "_annotations.coco.json")
    id_to_image, coco_anns = load_coco_annotations(coco_json)
    
    # Create output directories
    predicted_dir = os.path.join(OUTPUT_DIR, "predicted")
    wrong_dir = os.path.join(OUTPUT_DIR, "wrong")
    os.makedirs(predicted_dir, exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "iou_threshold"), exist_ok=True)
    os.makedirs(os.path.join(wrong_dir, "label_mismatch"), exist_ok=True)
    
    # Metrics tracking
    global_tp = 0
    global_fp = 0
    global_fn = 0
    failure_data = []
    
    # Process each image
    for image_id in tqdm(id_to_image, desc="Processing Images"):
        img_info = id_to_image[image_id]
        img_path = os.path.join(COCO_TEST_DIR, img_info["file_name"])
        
        # Load image and get predictions
        image = Image.open(img_path)
        detections = model.predict(image)
        pred_boxes = np.array(detections.xyxy)
        
        # Get ground truth boxes
        gt_boxes = []
        if image_id in coco_anns:
            for ann in coco_anns[image_id]:
                x, y, w, h = ann["bbox"]
                gt_boxes.append([x, y, x+w, y+h])
        gt_boxes = np.array(gt_boxes)
        
        # Create annotated image
        processed_img = cv2.imread(img_path)
        filename = os.path.basename(img_path)
        
        # Draw annotations
        if gt_boxes.size > 0:
            for gt in gt_boxes:
                cv2.rectangle(processed_img, 
                            (int(gt[0]), int(gt[1])),
                            (int(gt[2]), int(gt[3])),
                            (0, 255, 0), 2)
        
        if pred_boxes.size > 0:
            for box in pred_boxes:
                cv2.rectangle(processed_img,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            (0, 0, 255), 2)
        
        # Save to predicted folder
        cv2.imwrite(os.path.join(predicted_dir, filename), processed_img)
        
        # Initialize metrics for current image
        tp = 0
        fp = 0
        fn = 0
        
        has_gt = gt_boxes.size > 0
        has_pred = pred_boxes.size > 0
        
        if has_gt and has_pred:
            cost_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))
            for i, gt in enumerate(gt_boxes):
                for j, pred in enumerate(pred_boxes):
                    cost_matrix[i, j] = -compute_iou(gt, pred)
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            
            matched_gt = set()
            matched_pred = set()
            
            for i, j in zip(row_ind, col_ind):
                iou_val = compute_iou(gt_boxes[i], pred_boxes[j])
                if iou_val >= IOU_THRESHOLD:
                    tp += 1
                    matched_gt.add(i)
                    matched_pred.add(j)
            
            fp += len(pred_boxes) - len(matched_pred)
            fn += len(gt_boxes) - len(matched_gt)
            
        elif has_pred:
            fp += len(pred_boxes)
        elif has_gt:
            fn += len(gt_boxes)
        
        # Update global metrics
        global_tp += tp
        global_fp += fp
        global_fn += fn
        
        # Failure analysis
        label_mismatch = (len(gt_boxes) != len(pred_boxes)) or (fp > 0) or (fn > 0)
        low_iou = (tp > 0) and (fp + fn > 0)
        
        # Save failure cases
        failure_reasons = []
        if label_mismatch:
            failure_reasons.append("label_mismatch")
            cv2.imwrite(os.path.join(wrong_dir, "label_mismatch", filename), processed_img)
        if low_iou:
            failure_reasons.append("iou_threshold")
            cv2.imwrite(os.path.join(wrong_dir, "iou_threshold", filename), processed_img)
        
        if failure_reasons:
            failure_data.append({
                "image": filename,
                "tp": tp,
                "fp": fp,
                "fn": fn,
                "failure_reasons": ", ".join(failure_reasons)
            })

    # Calculate final metrics
    precision, recall = calculate_metrics(global_tp, global_fp, global_fn)
    
    # Save metrics to file
    metrics_content = f"""Evaluation Metrics:
    - True Positives (TP): {global_tp}
    - False Positives (FP): {global_fp}
    - False Negatives (FN): {global_fn}
    - Precision: {precision:.4f}
    - Recall: {recall:.4f}
    - F1 Score: {2*(precision*recall)/(precision+recall):.4f}"""
    
    with open(os.path.join(OUTPUT_DIR, "metrics.txt"), "w") as f:
        f.write(metrics_content)
    
    # Save CSV report
    df = pd.DataFrame(failure_data)
    csv_path = os.path.join(OUTPUT_DIR, "failure_report.csv")
    df.to_csv(csv_path, index=False)
    
    # Create grids for wrong predictions
    for subfolder in ["iou_threshold", "label_mismatch"]:
        subfolder_path = os.path.join(wrong_dir, subfolder)
        if os.path.exists(subfolder_path):
            image_files = sorted([f for f in os.listdir(subfolder_path) 
                               if f.endswith('.jpg') and not f.startswith('grid_')])
            
            for i in range(0, len(image_files), 16):
                chunk = image_files[i:i+16]
                grid_path = os.path.join(subfolder_path, f"grid_{i//16 + 1}.jpg")
                image_paths = [os.path.join(subfolder_path, f) for f in chunk]
                create_image_grid(image_paths, grid_path)

    # Create zip archive
    shutil.make_archive(ZIP_PATH.rstrip('.zip'), 'zip', OUTPUT_DIR)
    
    print(f"\n✅ Analysis complete!")
    print(f"📦 Download results: {ZIP_PATH}")
    print(metrics_content)

if __name__ == "__main__":
    analyze_predictions()

Model exists: True
Loading pretrain weights


Processing Images: 100%|██████████| 348/348 [00:15<00:00, 23.03it/s]



✅ Analysis complete!
📦 Download results: /kaggle/working/results-regular.zip
Evaluation Metrics:
    - True Positives (TP): 383
    - False Positives (FP): 386
    - False Negatives (FN): 102
    - Precision: 0.4980
    - Recall: 0.7897
    - F1 Score: 0.6108
